# 2. Tiedon yhdistely

## 2.1 Taustaa

Kunnilta saatavat lähtötiedot (kaava-aineistot) voivat vaihdella sisällöiltään valtavasti. Stabiilin vertailun ja validoinnin mahdollistamiseksi ETL-työkalussa luodaan sekä KTJ- että kunta-aineistoille yhdistetty kaava-aineisto yhtenäisellä skeemalla. Tiedot tallennetaan omina tasoinaan yhteisen master-geopackagen alle. Tasot ovat seuraavat:

- Asemakaavat_kunta
- Asemakaavat_ktj
- Yleiskaavat_kunta
- Yleiskaavat_ktj

**ETL-työkalussa asemakaava-tasot kattavat sekä asema- että ranta-asemakaavat!**

Kaavalajit on ilmoitettu KTJ-aineistossa valmiina [Kiinteistötietojärjestelmän kiinteistörekisterin koodistojen](https://www.maanmittauslaitos.fi/sites/maanmittauslaitos.fi/files/old/KR-koodistokokonaan_20111006_0.pdf) mukaisesti. Ne muunnetaan ETL-työkalussa vastaamaan [Rakennetun ympäristön tietojärjestelmän koodeja](https://koodistot.suomi.fi/codescheme;registryCode=rytj;schemeCode=RY_Kaavalaji).

Kuntien aineistoissa kaavalajeja ei ole välttämättä ilmoitettu ollenkaan erillisellä ominaisuustietokentällä. Tästä syystä ETL-päättelee nämä aineiston kaavaselitteistä. Mikäli selitteitä ei ole aineistoissa kerrottu, kaavalajeiksi ilmoitetaan oletuksena:

- Yleiskaavoilla 23 = osayleiskaava,
- Ranta-asemakaavoilla 33 = ranta-asemakaava ja
- Asemakaavoilla 31 = asemakaava.

Yhdistetyn kaava-aineiston skeema on esitelty alla olevassa taulukossa. Rasti kuvaa kunta- ja KTJ-aineiston osalta sitä, onko ominaisuustieto mukana aineiston skeemassa vai ei.

| Ominaisuustieto | Selite | Kunta-aineisto | KTJ-aineisto |
| :--- | :--- | :--- | :--- |
| FID | Rivin yksilöivä tunniste | X | X |
| originalref | Lähtöaineiston alkuperäinen koordinaattijärjestelmä epsg-koodina | X | X |
| vanhakuntakoodi | Kuntaliitosalueiden vanha kuntakoodi |  | X |
| kuntakoodi | Tilastokeskuksen mukainen voimassa oleva kuntakoodi | X | X |
| kuntanimi | Kunnan nimi | X | X |
| kaavatunnus | Kunnan antama kaavatunnus | X |  |
| kaavatunnus_1 | KTJ-aineiston kaavatunnuksen alkuosa |  | X |
| kaavatunnus_2 | KTJ-aineiston kaavatunnuksen loppuosa |  | X |
| kaavaselite | Kaavan tai kaavaindeksin selite/nimi | X | X |
| kaavalaji | Rakennetun ympäristön tietojärjestelmän koodistojen mukainen kaavalajikoodi | X | X |
| hyvaksymispvm | Kaavan hyväksymispäivämäärä | X | X |
| vahvistamispvm | Kaavan vahvistamispäivämäärä | X | X |
| voimaantulopvm | Kaavan voimaantulopäivämäärä | X | X |
| kohderekisteriyksikot | Lista kaavaindeksiin liittyvistä kiinteistötunnuksista | X | X |
| kaavakartta | Kaavakartan PDF-liitteen nimi tai hyperlinkki | X | X |
| maaraykset | Kaavamääräysten PDF-liitteen nimi tai hyperlinkki | X | X |
| selostus | Kaavaselostuksen PDF-liitteen nimi tai hyperlinkki | X | X |

**Yhdistetty kaavadata on keskeisessä osassa kaikissa ETL-työkalun vaiheissa, sillä koko toimintalogiikka perustuu muodostetun master-datasetin skeemaan!**

## 2.2 Kohderekisteriyksiköiden linkittäminen

Kiinteistötunnuksien sijaintitiedot on saatavilla MML:n kiinteistötietojen OGC API Features -rajapinnasta omana pistetasonaan (katso 1-tiedonkeruu-ja-esikasittely.ipynb). ETL-muunnin linkittää jokaiselle kaavaindeksille tiedon siihen liittyvistä kiinteistötunnuksista, mikäli yksittäisen kiinteistötunnuksen sijaintipiste on kaavaindeksin ulkorajojen sisällä. **Kohderekisteriyksiköiden linkitys kaavadataan tapahtuu osana yhdistetyn kaavadatan luomista (2.3.1 ja 2.3.2)!**

## 2.3 Yhdistetty kaavadata

### 2.3.1 KTJ master geopackage

In [ ]:
# Käytettävien Python moduulien ja kehitettyjen funktioiden sisäänluku
import geopandas as gpd
import pickle
from lib.kiinteistotunnus_to_kaavaindex import kiinteistotunnusToKaavaIndex
from lib.ktj_to_master_geopackage import setupKTJMasterDataframe, appendKTJToMaster
from lib.accessory_functions import saveGPKG

In [ ]:
# KTJ kaavadata. Lisää parametreiksi tarvittavat tiedot!
ktj = gpd.read_file(r"<insert filepath here>.gpkg", layer="<insert layer_name here>")

# Kohderekisteriyksiköt
with open(r"<insert filepath here>.pkl", "rb") as f:
    kiinttunnukset = pickle.load(f)

# Prosessointi
# Käsittele aineistot kunta kerrallaan!
kunta_ktj = ktj.loc[ktj['validkuntanro'] == '<insert kuntanro here>']
kunta_ktj_yhd = kiinteistotunnusToKaavaIndex(kiinttunnus_data=kiinttunnukset, kaava_data=kunta_ktj)

# Aja vain kerran ennen ensimmäistä appendia!
master_df = setupKTJMasterDataframe()

master_df = appendKTJToMaster(masterdf=master_df, ktjdata=kunta_ktj_yhd, geometry='geometry') 

# Tallennus
asemakaavat = [31, 32, 33, 34, 35, 39]
yleiskaavat = [21, 22, 23, 24, 25, 26]

asema = master_df.loc[master_df['kaavalaji'].isin(asemakaavat)]
yleis = master_df.loc[master_df['kaavalaji'].isin(yleiskaavat)]

saveGPKG(asema, outputfp=r"<insert filepath here>.gpkg", layer_name="asemakaavat_ktj")
saveGPKG(yleis, outputfp=r"<insert filepath here>.gpkg", layer_name="yleiskaavat_ktj")

### 2.3.2 Kunta master geopackage

Ominaisuustiedot vaihtelevat kunta-aineistoissa valtavasti. Tästä syystä suuri osa master-geopackagen kentistä on annettu appendKuntaToMaster-funktiolle valinnaisena parametrina. Mikäli jotain tietoa ei kunta-aineistosta löydy, kyseisen parametrin voi jättää antamatta funktiolle, jolloin dataan ilmeistyy riveille tieto None. Jos taas tieto löytyy, appendKuntaToMaster-funktiolle syötetään kunta-aineiston vastaavan kentän nimi parametrina.

In [ ]:
# Kehitettyjen funktioiden sisäänluku
from lib.kunta_to_master_geopackage import setupKuntaMasterDataframe, appendKuntaToMaster

In [ ]:
# Kunta kaavadata. Lisää parametriksi tarvittavat tiedot!
kunta_kaavadata = gpd.read_file(r"<insert filepath here>.shp")

# Kohderekisteriyksiköiden linkittäminen
kunta_kaavadata_yhd = kiinteistotunnusToKaavaIndex(kiinttunnus_data=kiinttunnukset, kaava_data=kunta_kaavadata)

# Aja vain kerran ennen ensimmäistä appendia!
master_df = setupKuntaMasterDataframe()

# Lisää tarvittavat parametrit! Lisätietoja kunta_to_master_geopackage.py docstringissa*!
master_df = appendKuntaToMaster(masterdf=master_df,
                                kaavadata=kunta_kaavadata_yhd,
                                kaavalaji='<insert column name here>',
                                geometry='<insert column name here>',
                                kuntakoodi='<insert column name here>',
                                kuntanimi='<insert column name here>',
                                kaavaselite='<insert column name here>')  # Valinnainen parametri (myös muita*!)

# Tallennus
asema = master_df.loc[master_df['kaavalaji'].isin(asemakaavat)]
yleis = master_df.loc[master_df['kaavalaji'].isin(yleiskaavat)]

saveGPKG(asema, outputfp=r"<insert filepath here>.gpkg", layer_name="asemakaavat_kunta")
saveGPKG(yleis, outputfp=r"<insert filepath here>.gpkg", layer_name="yleiskaavat_kunta")